# CNEFE 2010 → Faces de Quadra
### Ana Luisa Maffini

In [1]:

from pathlib import Path
TXT_PATH = Path("C:/Users/Ana Maffini/Downloads/Código CNEFE 2010/43000340500/43000340500.TXT")     # <- ajuste se necessário
FACE_SHP = Path("C:/Users/Ana Maffini/Downloads/Código CNEFE 2010/43000340500/43000340500_face.shp")# <- ajuste se necessário
OUT_DIR  = Path("C:/Users/Ana Maffini/Downloads/Código CNEFE 2010/output"); OUT_DIR.mkdir(parents=True, exist_ok=True)
print("TXT existe:", TXT_PATH.exists(), "| SHP existe:", FACE_SHP.exists(), "| OUT_DIR:", OUT_DIR)


TXT existe: True | SHP existe: True | OUT_DIR: C:\Users\Ana Maffini\Downloads\Código CNEFE 2010\output


In [2]:

import pandas as pd, numpy as np, geopandas as gpd, re
pd.set_option("display.max_columns", 80)


## 1) Leitura conforme especificação fornecida

In [3]:

colspecs = [
    (0, 2),    # Código da UF
    (2, 7),    # Código do município
    (7, 9),    # Código do distrito
    (9, 11),   # Código do subdistrito
    (11, 15),  # Código do setor
    (15, 16),  # Situação do setor
    (16, 36),  # Tipo do logradouro
    (36, 66),  # Título do logradouro
    (66, 126), # Nome do logradouro
    (126, 134),# Número do logradouro
    (134, 141),# Modificador do número
    (141, 161),# Elemento 1
    (161, 171),# Valor 1
    (171, 191),# Elemento 2
    (191, 201),# Valor 2
    (201, 221),# Elemento 3
    (221, 231),# Valor 3
    (231, 251),# Elemento 4
    (251, 261),# Valor 4
    (261, 281),# Elemento 5
    (281, 291),# Valor 5
    (291, 311),# Elemento 6
    (311, 321),# Valor 6
    (321, 336),# Latitude
    (336, 351),# Longitude
    (351, 411),# Localidade
    (411, 471),# Nulo
    (471, 473),# Espécie de endereço (1..7)
    (473, 513),# Identificação estabelecimento (descrição)
    (513, 514),# Indicador de endereço
    (514, 544),# Identificação domicílio coletivo
    (544, 547),# Número da quadra
    (547, 550),# Número da face
    (550, 558),# CEP
]
column_names = [
    "cod_uf","cod_municipio","cod_distrito","cod_subdistrito","cod_setor","situacao_setor",
    "tipo_logradouro","titulo_logradouro","nome_logradouro","numero_logradouro","modificador_numero",
    "elemento1","valor1","elemento2","valor2","elemento3","valor3","elemento4","valor4","elemento5","valor5",
    "elemento6","valor6","latitude","longitude","localidade","nulo","especie_uso","identiEstab",
    "indicador_endereco","identi_domic_coletivo","num_quadra","num_face","cep"
]

# Leitura única (um arquivo)
CNEFE_2010 = pd.read_fwf(TXT_PATH, colspecs=colspecs, names=column_names, encoding="latin1")
print("Linhas lidas:", len(CNEFE_2010))
display(CNEFE_2010.head(3))


Linhas lidas: 658


,cod_uf,cod_municipio,cod_distrito,cod_subdistrito,cod_setor,situacao_setor,tipo_logradouro,titulo_logradouro,nome_logradouro,numero_logradouro,modificador_numero,elemento1,valor1,elemento2,valor2,elemento3,valor3,elemento4,valor4,elemento5,valor5,elemento6,valor6,latitude,longitude,localidade,nulo,especie_uso,identiEstab,indicador_endereco,identi_domic_coletivo,num_quadra,num_face,cep
0,43,34,5,0,1,1,RUA,NaN,507,305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BELA VISTA,NaN,1,NaN,NaN,NaN,8,4,96445000
1,43,34,5,0,1,1,RUA,NaN,507,349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BELA VISTA,NaN,1,NaN,NaN,NaN,8,4,96445000
2,43,34,5,0,1,1,RUA,NaN,507,0,SN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BELA VISTA,NaN,6,ESTREVARIA,1.0,NaN,8,4,96445000


## 2) Padronização (zero‑fill) e construção dos IDs

In [4]:

lengths = {
    "cod_uf": 2, "cod_municipio": 5, "cod_distrito": 2, "cod_subdistrito": 2, "cod_setor": 4,
    "num_quadra": 3, "num_face": 3
}

for col, L in lengths.items():
    CNEFE_2010[col] = (CNEFE_2010[col].astype(str)
                                      .str.replace(r"\D","", regex=True)
                                      .str.zfill(L))

CNEFE_2010["ID_municipio"] = CNEFE_2010["cod_uf"] + CNEFE_2010["cod_municipio"]
CNEFE_2010["ID_setor"] = (CNEFE_2010["cod_uf"] + CNEFE_2010["cod_municipio"] +
                          CNEFE_2010["cod_distrito"] + CNEFE_2010["cod_subdistrito"] +
                          CNEFE_2010["cod_setor"])
CNEFE_2010["ID_face"] = (CNEFE_2010["cod_uf"] + CNEFE_2010["cod_municipio"] +
                         CNEFE_2010["cod_distrito"] + CNEFE_2010["cod_subdistrito"] +
                         CNEFE_2010["cod_setor"] + CNEFE_2010["num_quadra"] + CNEFE_2010["num_face"])

print("Exemplos ID_face:", CNEFE_2010["ID_face"].head().tolist())
print("Comprimento ID_face (esperado 21):", CNEFE_2010["ID_face"].str.len().value_counts().head())


Exemplos ID_face: ['430003405000001008004', '430003405000001008004', '430003405000001008004', '430003405000001008004', '430003405000001008001']
Comprimento ID_face (esperado 21): ID_face
21    658
Name: count, dtype: int64


## 3) Classificação (especie_uso 1..7) e refinamento do código 6

In [ ]:

MAP_CNEFE = {
    1: 'Residencial',
    2: 'Serviços publicos/adm.',
    3: 'Ensino',
    4: 'Saude',
    5: 'Equipamentos coletivos',
    6: 'Outros estabelecimentos',
    7: 'Nao residencial (outros)'
}

SUBMAP_OUTROS = {
    'mercado|supermerc|merceria|armazem|minimerc': 'Mercado',
    'supermercado|hipermercado': 'Supermercado',
    'padaria|padarria|padar': 'Padaria/Confeitaria',
    'acoug|açoug|açougue|assougue': 'Acougue',
    'farmac|drogari|farmácia|drogaria|farmacia': 'Farmacia',
    'bar|pub|botec|barr|boteco': 'Bar',
    'restaur|lanchon|cafe|lancheria|lanchonete|': 'Restaurante/Lancheria',
    'cafeter|cafeteria': 'Cafeteria',
    'auditorio|arena|show|boliche': 'Lazer',
    'biblioteca|museu|teatro|centro cultural|galeria|galeria de arte' : 'Bibliotecas/Museus',
    'clube|gremio|clube despotivo|clube esportivo|ginasio|ginásio|campo de futebol|campo de golfe|escolinha de futebol|haras|hípica|hipica|qudra': 'Centro Esportivo',
    'academia|pilates|yoga|ioga|ginastica|ginástica': 'Academia',
    'estacionamento|estacionamentos|estaconamento|garagem|estacionamentp': 'Estacionamento',
    'terminal|aeroporto|ônibus|onibus|taxi|aerporto': 'Transportes',
    'cemiterio|crematorio|cemitério|crematório': 'Cemiterio/Crematorio',
    'correio|correios|servicos postais|serviços postais|telecom|central telefonica|telefonia|telefonica|embratel': 'Correios/Telecomunicacoes',
    'hotel|hostel|pousada|pensao|pensão|motel|pousad|hosped': 'Hotelaria',
    'asilo|casa idosos|casa de repouso|moradia estudantil': 'Instiruicao Residencial',
    'hospital|casa de saude|casa de saúde': 'Hospital',
    'laboratório|laboratorio|analises clinicas|análises clínicas': 'Laboratorio',
    'banco|caixa|lotéric|loterica|itau|bradesco|banrisul': 'Servicos financeiros',
    'oficina|mecan|auto|borrachar|borracharia': 'Oficina/Servicos automotivos',
    'loja|comerci|roupa|calcad|move': 'Comercio varejista',
    'igreja|templo|paroqu|assembleia': 'Religioso',
    'posto\\s*(de)?\\s*saud|ubs|clinic| ambulatorio|ambulatório|posto de saude|posto de saúde|consultorio|clinica|policlinica|clínica|policlínica|emergência| unimed|': 'Saude',
    'escritori|cartori|corretor|imobili': 'Serviços profissionais',
    'escola|creche|colegio|univers': 'Ensino privado',
    'deposit|galp|garag|armaz|dep[oó]sit': 'Deposito/Galpão',
    'ferro\\s*velho|sucata': 'Reciclagem/Sucatas',
    'praca|pracas|praça|praças|parque|area verde|pracinha|bosque|cachoeira|parque municipal': 'Areas Verdes'
}

def refine_outros(desc: str) -> str:
    if not isinstance(desc, str) or not desc.strip():
        return 'Outros (sem descrição)'
    t = desc.lower()
    for pat, label in SUBMAP_OUTROS.items():
        if re.search(pat, t):
            return label
    return 'Outros (não mapeado)'

CNEFE_2010["especie_uso_num"] = pd.to_numeric(CNEFE_2010["especie_uso"], errors="coerce").astype("Int64")
CNEFE_2010["cat_principal"] = CNEFE_2010["especie_uso_num"].map(MAP_CNEFE).fillna("Desconhecido")
CNEFE_2010["cat_refinada"] = np.where(
    CNEFE_2010["especie_uso_num"]==6,
    CNEFE_2010["identiEstab"].fillna("").apply(refine_outros),
    CNEFE_2010["cat_principal"]
)
CNEFE_2010[["especie_uso","identiEstab","cat_principal","cat_refinada"]].head(8)


,especie_uso,identiEstab,cat_principal,cat_refinada
0,1,NaN,Residencial,Residencial
1,1,NaN,Residencial,Residencial
2,6,ESTREVARIA,Outros estabelecimentos,Outros (não mapeado)
3,1,NaN,Residencial,Residencial
4,1,NaN,Residencial,Residencial
5,6,GALPAO,Outros estabelecimentos,Deposito/Galpão
6,6,GALPAO,Outros estabelecimentos,Deposito/Galpão
7,1,NaN,Residencial,Residencial


## 4) Agregação por face

In [10]:

def aggregate_by_face(df: pd.DataFrame) -> pd.DataFrame:
    pivot = (df.assign(n=1)
               .pivot_table(index="ID_face", columns="cat_refinada", values="n", aggfunc="sum", fill_value=0))
    pivot.columns = [f"n_{re.sub('[^A-Za-z0-9]+','_',c).strip('_').lower()}" for c in pivot.columns]
    totals = df.groupby(["ID_face","cat_principal"]).size().unstack(fill_value=0)
    totals.columns = [f"tot_{re.sub('[^A-Za-z0-9]+','_',c).strip('_').lower()}" for c in totals.columns]
    return pivot.join(totals, how="outer").reset_index()

por_face = aggregate_by_face(CNEFE_2010)
print("por_face linhas:", len(por_face))
display(por_face.head(3))


por_face linhas: 146


,ID_face,n_bar,n_comercio_de_alimentos,n_comercio_varejista,n_deposito_galp_o,n_equipamentos_coletivos,n_farmacia,n_nao_residencial_outros,n_oficina_servicos_automotivos,n_outros_n_o_mapeado,n_reciclagem_sucatas,n_religioso,n_residencial,n_restaurante_lancheria,n_saude,n_servicos_financeiros,n_servi_os_profissionais,n_servi_os_publicos_adm,tot_equipamentos_coletivos,tot_nao_residencial_outros,tot_outros_estabelecimentos,tot_residencial,tot_saude,tot_servi_os_publicos_adm
0,430003405000001001002,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
1,430003405000001001003,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,2,0,0
2,430003405000001003001,1,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,1,3,0,0


## 5) Join com SHP de faces (busca automática de coluna e sufixo ótimo)

In [11]:

faces = gpd.read_file(FACE_SHP)

# colunas candidatas típicas
cand = [c for c in faces.columns if re.search(r'cd_?geo|geocod|face|cod', c, flags=re.I)] or list(faces.columns)

def only_digits(s):
    return re.sub(r"\D","", str(s) if s is not None else "")

def build_suffix(series, L):
    s = series.astype(str).map(only_digits)
    return s.apply(lambda x: x[-L:] if len(x)>=L else None)

# tente casar usando diferentes comprimentos (ID_face tipicamente 21; mas alguns SHPs armazenam 14/15/17/19)
L_candidates = sorted({21, por_face["ID_face"].str.len().mode().iat[0], 14, 15, 17, 19, 20, 22})
results = {}
best = {"col": None, "L": None, "ratio": -1.0, "key": None}
for col in cand:
    for L in L_candidates:
        key = build_suffix(faces[col], L)
        ratio = key.isin(por_face["ID_face"]).mean()
        results[(col,L)] = ratio
        if ratio > best["ratio"]:
            best = {"col": col, "L": L, "ratio": ratio, "key": key}

print("Melhor coluna/L encontrados:", best["col"], best["L"], "| match:", round(best["ratio"],3))
faces["ID_face"] = best["key"]

matched = faces["ID_face"].isin(por_face["ID_face"]).sum()
print(f"Faces com match: {matched} de {len(faces)}")

joined = faces.merge(por_face, on="ID_face", how="left")
print("Joined shape:", joined.shape)
joined.head(2)


Melhor coluna/L encontrados: CD_GEO 21 | match: 0.629
Faces com match: 146 de 232
Joined shape: (232, 35)


,ID,CD_GEO,CD_SETOR,CD_QUADRA,CD_FACE,NM_TIPO_LO,NM_TITULO_,NM_NOME_LO,TOT_RES,TOT_GERAL,geometry,ID_face,n_bar,n_comercio_de_alimentos,n_comercio_varejista,n_deposito_galp_o,n_equipamentos_coletivos,n_farmacia,n_nao_residencial_outros,n_oficina_servicos_automotivos,n_outros_n_o_mapeado,n_reciclagem_sucatas,n_religioso,n_residencial,n_restaurante_lancheria,n_saude,n_servicos_financeiros,n_servi_os_profissionais,n_servi_os_publicos_adm,tot_equipamentos_coletivos,tot_nao_residencial_outros,tot_outros_estabelecimentos,tot_residencial,tot_saude,tot_servi_os_publicos_adm
0,8056279,430003405000001001003,430003405000001,001,003,RODOVIA,None,BR 153,2.0,2.0,"LINESTRING (-54.16678 -31.86346, -54.16655 -31...",430003405000001001003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,8056242,430003405000001010004,430003405000001,010,004,RODOVIA,None,BR 153,0.0,1.0,"LINESTRING (-54.16653 -31.86593, -54.1664 -31....",430003405000001010004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## 6) Exportar resultados

In [12]:

gpkg = OUT_DIR / "cnefe_faces_fwf.gpkg"
csvf = OUT_DIR / "cnefe_por_face_fwf.csv"

# Evita future warning de downcast silencioso
joined = joined.infer_objects(copy=False).fillna(0)

joined.to_file(gpkg, layer="faces", driver="GPKG")
por_face.to_csv(csvf, index=False)

print("Arquivos gerados:")
print("-", gpkg)
print("-", csvf)


Arquivos gerados:
- C:\Users\Ana Maffini\Downloads\Código CNEFE 2010\output\cnefe_faces_fwf.gpkg
- C:\Users\Ana Maffini\Downloads\Código CNEFE 2010\output\cnefe_por_face_fwf.csv


C:\Users\Ana Maffini\AppData\Local\Temp\ipykernel_6188\1626005227.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  joined = joined.infer_objects(copy=False).fillna(0)
